In [1]:
import pandas as pd
import numpy as np

In [ ]:
notes = pd.read_csv('~/payload/mimicdata/NOTEEVENTS.csv', low_memory=False)

In [1]:
##Note: each subject_id might correspond to multiple visits to hospital 

In [2]:
notes = notes[notes['ISERROR'] != 1]
notes.shape

In [3]:
notes = notes[notes['CATEGORY'] != 'Discharge summary'] ## drop discharge notes
notes = notes[notes['CATEGORY'] != 'Nursing/other']
notes.shape

NameError: name 'notes' is not defined

In [73]:
## There are 46146 patients, 2083180 recordings
data = notes[['SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'TEXT']]

In [75]:
##Note: group by visits, otherwise would be "cheating": predict a patient's LoS based on
## their fourth visits 
visits = data.groupby('HADM_ID')
#counted and got only 58361 visits 
def preprocess_notes(vst, max_len):
    vst = vst.sort_values('CHARTDATE')  #arrange by day1,day2, day3..
    #input VST is a dataframe of all rows in each individual visit
    # return a new series with shorter sentences but corresponds to each day during this visit
    df = vst['TEXT'] ##extract "TEXT" column as a series
    df = df.str.replace('\n', ' ')
    df = df.str.replace('  ', ' ')
    df = df.str.replace('\*+', '')
    df = df.str.replace('\d+', '')
    df = df.str.replace('_', '')
    df = df.str.replace('-', '')
    df = df.str.strip("[]")
    #print(df)
    newcol = []
    for row in df:
        short = ''
        lst = row.split(':')
        lst.sort(key=len)
        #print(lst)
        while len(short) < max_len and lst != []:
            short += lst.pop()
        #short = short[0:max_len]  does each sentece have to be 512?
        newcol.append(short)
    newcol = pd.DataFrame({'NOTE': newcol}) 
    
    result = vst[['SUBJECT_ID', 'HADM_ID', 'CHARTDATE']]
    newcol= newcol.set_index(result.index)
    
    #newcol.apply(lambda x:str(x))
    #print(type(newcol.iloc[0]))
    result = pd.concat([result, newcol], axis=1)
    result = result.set_index('HADM_ID')
    return result

In [77]:
c = 0
prsd = pd.DataFrame({'SUBJECT_ID':[], 'CHARTDATE':[], 'NOTE':[]})
prsd
for key, val in visits:
    #print(val)
    pro = preprocess_notes(val, 512)
    prsd = pd.concat([prsd, pro])
    c += 1

prsd.shape
## for each stay, we don't want repeated info over a span of days

(969555, 3)

In [79]:
prsd.to_csv(r'processed.csv')

In [63]:
## each person has multiple days of stay: we take 1st day, 3rd, and 6th day notes?
max_len = 512
processed = 
for key, val in visits:
    pro = preprocess_notes(val, 512)
    yo = pro
    break
for key, val in notes:
    pro = preprocess_notes(val, 500)
    yo = yo.append(pro, ignore_index=True)
    print(8)
    
yo

SyntaxError: invalid syntax (<ipython-input-63-0faa062bb22a>, line 3)

In [83]:
example = vis.iloc[2]['TEXT']
len(example)
print(example)

[**2124-7-21**] 10:57 AM
 CHEST (PORTABLE AP)                                             Clip # [**Clip Number (Radiology) 13062**]
 Reason: ?edema
 ______________________________________________________________________________
 [**Hospital 2**] MEDICAL CONDITION:
  87 year old woman with hx COPD, here with dyspnea acute x 2 hours, wheezing,
  also hx chf
 REASON FOR THIS EXAMINATION:
  ?edema
 ______________________________________________________________________________
                                 FINAL REPORT
 PORTABLE CHEST RADIOGRAPH PERFORMED ON [**2124-7-21**].

 CLINICAL HISTORY:  87-year-old woman with COPD, dyspnea, wheezing, question
 CHF, pneumonia.

 FINDINGS:  Portable AP upright view of the chest is obtained.  Evaluation is
 limited given motion blur artifact.  There is a large retrocardiac density
 containing gas compatible with hiatal hernia, as seen on prior.  The lungs
 appear grossly clear bilaterally.  Cardiomediastinal silhouette appears
 stable.  Bony struc

In [61]:
check1 = notes.groupby('CATEGORY')
check2 = notes.groupby('DESCRIPTION')
for name, group in check1:
    print(name)
    
#for name, group in check1:
    #print(name)
## category can use one-hot, but description should probably embed

Case Management 
Consult
ECG
Echo
General
Nursing
Nursing/other
Nutrition
Pharmacy
Physician 
Radiology
Rehab Services
Respiratory 
Social Work


In [52]:
yo = "I saw and examined the pt, and was present with the ICU In short this is a [**Age over 90 **] yo female"
len(yo)

103